In [1]:
import ifcopenshell
import ifcopenshell.geom

In [2]:
import OCC.Core.gp
import OCC.Core.Geom

import OCC.Core.Bnd
import OCC.Core.BRepBndLib

import OCC.Core.BRep
import OCC.Core.BRepPrimAPI
import OCC.Core.BRepAlgoAPI
import OCC.Core.BRepBuilderAPI

import OCC.Core.GProp
import OCC.Core.BRepGProp

import OCC.Core.TopoDS
import OCC.Core.TopExp
import OCC.Core.TopAbs

In [3]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

In [18]:
occ_display = ifcopenshell.geom.utils.initialize_display()

INFO:OCC.Display.backend:The qt-pyside2 backend is already loaded...``load_backend`` can only be called once per session
qt-pyside2 backend - Qt version 5.15.8
Display3d class successfully initialized.
#########################################
OpenGl information:
  GLvendor: NVIDIA Corporation
  GLdevice: NVIDIA GeForce RTX 4060 Ti/PCIe/SSE2
  GLversion: 4.6.0 NVIDIA 525.125.06
  GLSLversion: 4.60 NVIDIA
  Max texture size: 32768
  Max FBO dump size: 32768x32768
  Max combined texture units: 192
  Max MSAA samples: 32
  Viewport: 1023x767
  Window buffer: RGB8 ALPHA0 DEPTH24 STENCIL0
  ResolutionRatio: 1
  FBO buffer: GL_SRGB8_ALPHA8 GL_DEPTH24_STENCIL8
####### 3D rendering pipe initialisation #####
Display3d class initialization starting ...
Aspect_DisplayConnection created.
OpenGl_GraphicDriver created.
V3d_Viewer created.
AIS_InteractiveContext created.
V3d_View created
Xw_Window created.
TKOpenGl | Type: Other | ID: 0 | Severity: Medium | Message:
  OpenGl_Window::CreateWindow: win

In [5]:
path = './ifcspace.ifc'

In [6]:
f = ifcopenshell.open(path)

In [7]:
walls = f.by_type('IfcWall')
wall_shapes = []
cols = f.by_type('IfcColumn')
col_shapes = []
beams = f.by_type('IfcBeam')
beam_shapes = []
bbox = OCC.Core.Bnd.Bnd_Box()

In [17]:
for w in walls:
    shape = ifcopenshell.geom.create_shape(settings, w).geometry
    wall_shapes.append((w, shape))
    OCC.Core.BRepBndLib.brepbndlib_Add(shape, bbox)
    #ifcopenshell.geom.utils.display_shape(shape)

In [20]:
bbox_center = ifcopenshell.geom.utils.get_bounding_box_center(bbox)
occ_display.DisplayMessage(bbox_center, "Center", update=True)

half_spaces = []

for wall, shape in wall_shapes:
    exp = OCC.Core.TopExp.TopExp_Explorer(shape, OCC.Core.TopAbs.TopAbs_FACE)
    while exp.More():
        face = OCC.Core.TopoDS.topods.Face(exp.Current())
        exp.Next()
        surf = OCC.Core.BRep.BRep_Tool.Surface(face)
        assert surf.DynamicType().Name() == 'Geom_Plane'

        plane = OCC.Core.Geom.Handle_Geom_Plane_DownCast(surf)

        if plane.Axis().Direction().Z() == 0:
            face_bbox = OCC.Core.Bnd.Bnd_Box()
            OCC.Core.BRepBndLib.brepbndlib_Add(face, face_bbox)
            
            face_center = ifcopenshell.geom.utils.get_bounding_box_center(face_bbox).XYZ()
            face_normal = plane.Axis().Direction().XYZ()
            face_towards_center = bbox_center.XYZ() - face_center
            face_towards_center.Normalize()

            dot = face_towards_center.Dot(face_normal)
            if dot < -0.8:
                ifcopenshell.geom.utils.display_shape(face)
                face_plane = plane.Pln()
                new_face = OCC.Core.BRepBuilderAPI.BRepBuilderAPI_MakeFace(face_plane).Face()
                half_space = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeHalfSpace(new_face, bbox_center).Solid()
                half_spaces.append(half_space)

In [21]:
common_shape = OCC.Core.BRepPrimAPI.BRepPrimAPI_MakeBox(OCC.Core.gp.gp_Pnt(-0.1, -0.1, -0.6), OCC.Core.gp.gp_Pnt(9,9,5)).Solid()
com_shape = ifcopenshell.geom.utils.display_shape(common_shape)
ifcopenshell.geom.utils.set_shape_transparency(ais=com_shape, t=0.9)
for half_sp in half_spaces:
    common_shape = OCC.Core.BRepAlgoAPI.BRepAlgoAPI_Common(common_shape, half_sp).Shape()

box_shape = ifcopenshell.geom.utils.display_shape(common_shape)
ifcopenshell.geom.utils.set_shape_transparency(ais=box_shape, t=0.8)

In [22]:
ifcopenshell.geom.utils.main_loop()

INFO:OCC.Display.qtDisplay:key: code 16777251 not mapped to any function
